In [1]:
import numpy as np
import random
import math
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
from queue import Queue
import warnings
import sys
import json
import time
import types
import pickle
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import os

In [2]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [ ]:
class MapGenerator:
    def __init__(
        self,
        map_type: Literal["random", "multi_narrow", "maze"] = "random",
        map_size: Union[Tuple[int, int], Tuple[int, int, int]] = (50, 50),
        obstacle_percent: float = 0.2,
        min_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (2, 2),
        max_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (5, 5),
        max_obstacle_count: Optional[int] = None
    ):
        self.map_type = map_type
        self.map_size = map_size
        self.obstacle_percent = obstacle_percent
        self.obstacle_list: List[Tuple[int, ...]] = []
        self.min_size = min_obstacle_size
        self.max_size = max_obstacle_size
        self.max_count = max_obstacle_count
        self.is_3d = len(map_size) == 3
        self.map = self._init_map()
        self.start = None
        self.goal = None

    def _init_map(self):
        shape = self.map_size[::-1] if self.is_3d else (self.map_size[1], self.map_size[0])
        return np.zeros(shape, dtype=np.uint8)

    def generate(self, start: Tuple[int, ...], goal: Tuple[int, ...]) -> Dict[str, Union[np.ndarray, Tuple[int, ...], List[Tuple[int, ...]], Tuple[int, int]]]:
        self.start = start
        self.goal = goal
        tries = 0
        max_tries = 100

        while tries < max_tries:
            self.map = self._init_map()
            self.obstacle_list.clear()

            if self.map_type == "maze":
                self.map = self._generate_maze()
            elif self.map_type == "random":
                self._generate_random_obstacles(start, goal)
            elif self.map_type == "multi_narrow":
                self._generate_multi_narrow(start, goal)

            if self._path_exists(start, goal):
                return Map(
                    grid=self.map,
                    start=start,
                    goal=goal,
                    obstacles=self.obstacle_list,
                    size=self.map_size
                )
            tries += 1

        raise RuntimeError("Failed to generate a connected map after multiple attempts.")

    def _add_obstacle(self, coords: Tuple[int, ...]):
        self.obstacle_list.append(coords)

    def _path_exists(self, start, goal):
        visited = set()
        q = Queue()
        q.put(start)
        visited.add(start)

        dims = len(start)
        neighbors = [(-1,0), (1,0), (0,-1), (0,1)] if dims == 2 else \
                    [(-1,0,0),(1,0,0),(0,-1,0),(0,1,0),(0,0,-1),(0,0,1)]

        while not q.empty():
            node = q.get()
            if node == goal:
                return True

            for delta in neighbors:
                neighbor = tuple(node[i] + delta[i] for i in range(dims))
                if self._in_bounds(neighbor) and neighbor not in visited:
                    if self._is_free(neighbor):
                        visited.add(neighbor)
                        q.put(neighbor)
        return False

    def _in_bounds(self, p):
        if self.is_3d:
            z, y, x = p
            return 0 <= z < self.map.shape[0] and 0 <= y < self.map.shape[1] and 0 <= x < self.map.shape[2]
        else:
            y, x = p
            return 0 <= y < self.map.shape[0] and 0 <= x < self.map.shape[1]

    def _is_free(self, p):
        if self.is_3d:
            z, y, x = p
            return self.map[z, y, x] == 0
        else:
            y, x = p
            return self.map[y, x] == 0

    def _is_inside(self, point, x, y, z, ow, oh, od):
        px, py, *pz = point
        if z is None:
            return x <= px < x + ow and y <= py < y + oh
        else:
            pz = pz[0]
            return x <= px < x + ow and y <= py < y + oh and z <= pz < z + od

    def _generate_random_obstacles(self, start, goal):
        if self.is_3d:
            w, h, d = self.map_size
            total_voxels = w * h * d
            max_obs = int(total_voxels * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                od = random.randint(self.min_size[2], self.max_size[2])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                z = random.randint(0, d - od - 1)
                if np.any(self.map[z:z+od, y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, z, ow, oh, od) or self._is_inside(goal, x, y, z, ow, oh, od):
                    continue
                self.map[z:z+od, y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, z, ow, oh, od))
                count += ow * oh * od
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break
        else:
            w, h = self.map_size
            total_cells = w * h
            max_obs = int(total_cells * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                if np.any(self.map[y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, None, ow, oh, None) or self._is_inside(goal, x, y, None, ow, oh, None):
                    continue
                self.map[y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, ow, oh))
                count += ow * oh
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break

    def _generate_multi_narrow(self, start, goal):
        if self.is_3d:
            self._generate_random_obstacles(start, goal)
        else:
            w, h = self.map_size
            corridor_width = 2
            spacing = 6
            for i in range(spacing, h - spacing, spacing + corridor_width):
                self.map[i:i+spacing, :] = 1
            self._carve_corridor(start, goal)

    def _carve_corridor(self, start, goal):
        if not self.is_3d:
            y0, x0 = start[1], start[0]
            y1, x1 = goal[1], goal[0]
            for x in range(min(x0, x1), max(x0, x1)+1):
                self.map[y0, x] = 0
            for y in range(min(y0, y1), max(y0, y1)+1):
                self.map[y, x1] = 0

    def _generate_maze(self):
        width, height = self.map_size
        if width % 2 == 0: width += 1
        if height % 2 == 0: height += 1

        maze = np.ones((height, width), dtype=np.uint8)
        sx, sy = 1, 1
        maze[sy, sx] = 0

        walls = [(sx + dx, sy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                 if 0 < sx + dx < width and 0 < sy + dy < height]

        while walls:
            wx, wy = walls.pop(random.randint(0, len(walls) - 1))
            if maze[wy, wx] == 1:
                neighbors = [(wx + dx, wy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                             if 0 < wx + dx < width and 0 < wy + dy < height and maze[wy + dy, wx + dx] == 0]
                if len(neighbors) == 1:
                    nx, ny = neighbors[0]
                    maze[(wy + ny) // 2, (wx + nx) // 2] = 0
                    maze[wy, wx] = 0
                    for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]:
                        nx, ny = wx + dx, wy + dy
                        if 0 < nx < width and 0 < ny < height and maze[ny, nx] == 1:
                            walls.append((nx, ny))

        return maze

In [7]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [4]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [ ]:
gen = MapGenerator(map_type="random", map_size=(100, 100), min_obstacle_size=(5, 5), max_obstacle_size=(15, 15), obstacle_percent=0.32)
m = gen.generate(start=(1, 1), goal=(99, 99))
visualize_map_shapes(m.grid, obs=m.obstacles, start=m.start, goal=m.goal)

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (80,50), (50,50)
    

    grid[27:30, 52:70] = 1  
    obstacles.append((27, 52, 3, 18))
    grid[27:30, 30:48] = 1  
    obstacles.append((27, 30, 3, 18))
    
    grid[70:73, 30:70] = 1
    obstacles.append((70, 30, 3, 40))

    grid[30:70, 30:33] = 1
    obstacles.append((30, 30, 40, 3))
    grid[30:70, 67:70] = 1  
    obstacles.append((30, 67, 40, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,40), (90,10)
    


    grid[17:20, 20:80] = 1
    obstacles.append((17, 20, 3, 60))
    grid[37:40, 0:50] = 1
    obstacles.append((37, 0, 3, 50))
    grid[57:60, 20:100] = 1
    obstacles.append((57, 20, 3, 80))
    grid[77:80, 20:70] = 1
    obstacles.append((77, 0, 3, 80))

    grid[20:37, 47:60] = 1  
    obstacles.append((20, 47, 17, 3))
    grid[30:57, 62:65] = 1 
    obstacles.append((30, 62, 27, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((10, 80, 35, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((70, 80, 20, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [112]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,10), (90,80)
    
    grid[27:30, 20:80] = 1
    obstacles.append((27, 20, 3, 60))
    grid[47:50, 0:50] = 1
    obstacles.append((47, 0, 3, 50))
    grid[77:80, 20:100] = 1
    obstacles.append((77, 20, 3, 80))

    grid[30:47, 47:50] = 1  
    obstacles.append((30, 47, 17, 3))
    grid[50:77, 72:65] = 1 
    obstacles.append((50, 72, 27, 3))

    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [113]:
map_obj = generate_box_with_narrow_entry(width=100, height=100, entry_side="left", entry_pos=25, entry_width=2)


In [5]:
visualize_map_shapes(map_obj.grid, obs=map_obj.obstacles, start=map_obj.start, goal=map_obj.goal)

NameError: name 'visualize_map_shapes' is not defined

In [10]:
# MapIO.save_map(map_obj, "Maze_map_easy.pkl")

In [9]:
maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

Start: (29, 10)
Obstacles: 5
(100, 100)


In [ ]:
maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

In [11]:
narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (80, 50)
Obstacles: 5
(100, 100)


In [25]:
# 불러오기
multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)


In [10]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal)

In [14]:
class MultiMapBenchmarker:
    def __init__(
        self,
        maps: List[np.ndarray],
        name: str = "Algorithm",
        learning_mode = True,
        iter = 10,
        seed = 42,
    ):
        self.set_seed(seed)

        self.maps = maps
        self.name = name
        self.results_df: pd.DataFrame = pd.DataFrame()
        self.iter = iter
        self.learning_mode = learning_mode

        self.time_limit = 5.0
        self.success_limit = 0.8

    def run(self, algorithm) -> pd.DataFrame:
        results = []
        main_start_time = time.time()
        for i, map_ in enumerate(self.maps):
            print(f"Map {i+1}")
            for j in range(self.iter):
                start_time = time.time()
                try:
                    output = algorithm(map_)
                except Exception as e:
                    output = {"path": [], "nodes": [], "n_nodes": 0}
                end_time = time.time()


                if isinstance(output, PlannerResult):
                    path = output.path
                    nodes = output.nodes
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                elif isinstance(output, Dict):
                    path = output['path']
                    nodes = output['nodes']
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                else:
                    return None, None
                
                print(f"Iteration {j+1}: Time taken: {end_time - start_time:.4f} seconds, Success: {success}")

                results.append({
                    "map_id": i,
                    "iter" : j,
                    "algorithm": self.name,
                    "success": success,
                    "time_taken": end_time - start_time,
                    "num_nodes": num_nodes,
                    "path_length": path_length
                })

                time_taken_avg = np.mean([r["time_taken"] for r in results])
                success_avg = np.mean([r["success"] for r in results]) if len(results) > 6 else 1.0
                if time_taken_avg > self.time_limit:
                    print("Time taken Limit")
                    return None, None
                elif success_avg < self.success_limit:
                    print("Success Rate Limit")
                    return None, None

        print(f"Total time taken for all maps: {time.time() - main_start_time:.4f} seconds")

        self.results_df = pd.DataFrame(results)
        return self.results_df, self.get_avg()

    def _compute_path_length(self, path: List[Tuple[float, ...]]) -> float:
        if not path or len(path) < 2:
            return 0.0
        return sum(np.linalg.norm(np.array(path[i]) - np.array(path[i+1])) for i in range(len(path) - 1))
    
    def _is_path_valid(self, path: List[Tuple[float, ...]], map_) -> bool:
        if not path or len(path) < 2:
            return False
        
        if np.linalg.norm(np.array(path[0]) - np.array(map_.start)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.goal)) > 0.5:
            if np.linalg.norm(np.array(path[0]) - np.array(map_.goal)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.start)) > 0.5:
                return False
            
        is_3d = True if len(map_.size) > 2 else False
        for i in range(len(path) - 1):
            p1 = path[i]
            p2 = path[i + 1]
            if self._is_edge_in_obstacle(p1, p2, map_.obstacles, is_3d):
                return False
        return True
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

    def save_results(self, filename: str):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")
        self.results_df.to_csv(filename, index=False)

    def plot_metrics(self, metric: str = "time_taken"):
        if self.results_df.empty:
            raise RuntimeError("No results to plot. Run benchmark first.")
        if metric not in self.results_df.columns:
            raise ValueError(f"Invalid metric: {metric}")

        fig = px.bar(
            self.results_df,
            x="map_id",
            y=metric,
            color="success",
            title=f"{self.name} - {metric} per map",
            labels={"map_id": "Map ID", metric: metric.replace('_', ' ').title()}
        )
        fig.show()

    def get_avg(self):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")

        classic_summary = self.results_df.groupby('map_id').agg({
            'success': 'mean',           # 성공률
            'time_taken': 'mean',        # 평균 소요 시간
            'num_nodes': 'mean',      # 평균 노드 수
            'path_length': lambda x: x[x > 0].mean()  # path=0 제외한 평균 경로 길이
        }).rename(columns={
            'success': 'success_rate',
            'time_taken': 'time_avg',
            'num_nodes': 'num_nodes_avg',
            'path_length': 'path_length_avg'
        }).reset_index()
        return classic_summary
    
    @staticmethod
    def get_improvement(reference_result:pd.DataFrame, results:pd.DataFrame) -> pd.DataFrame:
        improvement_df = pd.DataFrame()
        
        improvement_df['success_improvement'] = (results['success_rate'] - reference_result['success_rate']) * 100 # percent point
        improvement_df['time_improvement'] = (results['time_avg'] - reference_result['time_avg']) / reference_result['time_avg'] * -100
        improvement_df['length_improvement'] = (results['path_length_avg'] - reference_result['path_length_avg']) / reference_result['path_length_avg'] * -100

        improvement_df['objective_score'] = (
            0.5 * improvement_df['success_improvement'] +
            0.3 * improvement_df['time_improvement'] +
            0.2 * improvement_df['length_improvement']
            )

        return improvement_df
    
    def set_seed(self, seed: int = 42):
        random.seed(seed)                    # Python random
        np.random.seed(seed)                 # NumPy

## Generate templete

In [ ]:
# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections


# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000):
        self.max_iter = max_iter

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

In [15]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'

In [282]:
result = {
    "algorithm": "Improved-RRT*-Connect",
    "algorithm_description": "Improved RRT*-Connect is a bidirectional, asymptotically optimal planner that enhances RRT*-Connect by incorporating informed heuristic sampling, adaptive step size, node rejection, and pruning. It accelerates convergence and improves success rate in narrow, obstacle-rich environments.",
    "planning_mechanism": "The planner grows two trees from start and goal using informed sampling. During expansion, it adaptively adjusts the step size near obstacles, rejects inefficient new nodes, and prunes branches that cannot contribute to an improved solution. The planner rewires nearby nodes only if doing so reduces path cost, and updates the current best path whenever a successful connection is found.",
    "code": '''
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

    def path_from_root(self):
        node, path = self, []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


class Planner:
    def __init__(self, max_iter=5000, step_size=5.0, rewire_radius=15.0):
        self.max_iter = max_iter
        self.base_step = step_size
        self.rewire_radius = rewire_radius

    def plan(self, map):
        import math, random, numpy as np

        bounds = map.size
        start, goal = map.start, map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3
        dim = len(bounds)

        tree_a, tree_b = [Node(start)], [Node(goal)]
        nodes = [tree_a[0], tree_b[0]]
        edges = []
        success, c_best, best_path = False, float("inf"), []
        c_min = math.dist(start, goal)

        for i in range(self.max_iter):
            tree_a, tree_b = (tree_a, tree_b) if i % 2 == 0 else (tree_b, tree_a)

            sample = self._informed_sample(start, goal, c_best, c_min, bounds, dim)
            nearest = min(tree_a, key=lambda n: math.dist(n.position, sample))

            step = self._adaptive_step(nearest.position, sample, obstacles, is_3d)
            new_pos = self._steer(nearest.position, sample, step)

            if self._is_in_obstacle(new_pos, obstacles, is_3d):
                continue
            if self._is_edge_in_obstacle(nearest.position, new_pos, obstacles, is_3d):
                continue

            cost = nearest.cost + math.dist(nearest.position, new_pos)
            if cost + math.dist(new_pos, goal) >= c_best:
                continue  # pruning

            new_node = Node(new_pos, nearest, cost)
            nearest.add_child(new_node)
            tree_a.append(new_node)
            nodes.append(new_node)
            edges.append((nearest, new_node))

            near_nodes = [n for n in tree_a if math.dist(n.position, new_node.position) <= self.rewire_radius]
            for near in near_nodes:
                new_cost = new_node.cost + math.dist(new_node.position, near.position)
                if new_cost < near.cost and not self._is_edge_in_obstacle(new_node.position, near.position, obstacles, is_3d):
                    if near.parent:
                        near.parent.children.remove(near)
                        edges.remove((near.parent, near))
                    near.parent = new_node
                    near.cost = new_cost
                    new_node.add_child(near)
                    edges.append((new_node, near))

            # Try to connect to the other tree
            other_nearest = min(tree_b, key=lambda n: math.dist(n.position, new_node.position))
            connect_cost = new_node.cost + math.dist(new_node.position, other_nearest.position) + other_nearest.cost
            if connect_cost < c_best and not self._is_edge_in_obstacle(new_node.position, other_nearest.position, obstacles, is_3d):
                c_best = connect_cost
                path_a = new_node.path_from_root()
                path_b = other_nearest.path_from_root()
                best_path = path_a + path_b[::-1]
                success = True

        return PlannerResult(success=success, path=best_path, nodes=nodes, edges=edges)

    def _informed_sample(self, start, goal, c_best, c_min, bounds, dim):
        import numpy as np, math, random
        if c_best == float("inf"):
            return tuple(random.uniform(0, bounds[d]) for d in range(dim))
        x_center = [(s + g) / 2 for s, g in zip(start, goal)]
        a1 = np.array(goal) - np.array(start)
        a1 = a1 / np.linalg.norm(a1)
        M = np.outer(a1, np.eye(dim)[0])
        U, _, Vt = np.linalg.svd(M)
        C = U @ np.diag([1] * (dim - 1) + [np.linalg.det(U) * np.linalg.det(Vt)]) @ Vt
        r1 = c_best / 2
        r2 = math.sqrt(c_best**2 - c_min**2) / 2
        L = np.diag([r1] + [r2] * (dim - 1))
        while True:
            x_ball = np.random.normal(0, 1, dim)
            x_ball /= np.linalg.norm(x_ball)
            x_ball *= random.random() ** (1 / dim)
            x_rand = C @ L @ x_ball + x_center
            if all(0 <= x_rand[d] <= bounds[d] for d in range(dim)):
                return tuple(x_rand)

    def _adaptive_step(self, from_pos, to_pos, obstacles, is_3d):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(2, int(distance))
        for i in range(1, steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return max(self.base_step * 0.3, 1.0)
        return self.base_step

    def _steer(self, from_pos, to_pos, step):
        import math
        dist = math.dist(from_pos, to_pos)
        if dist <= step:
            return to_pos
        return tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * step / dist for d in range(len(from_pos)))

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

    '''
}

# 저장
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)
    f.write(",\n")

In [16]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [ ]:
Instruction : Generate Improved-RRT*-Connect algorithm

Refer to below architecture:
!!!!!generate Only class Node and Planner!!!!!!
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float=5.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

### Objective:
- Improve path planning performance in terms of:
  - Planning efficiency
  - Path quality
  - Robustness
  - Success rate
  - Path smoothness
  - Path lengths
  - Reduce search time

### Constraints:
- Implement it in Python.
- You DO NOT NEED to declare the any imports.
- When connecting nodes and adding edges in the planner, always perform two critical checks:
1.Collision check for the node position: Ensure that the new node itself does not lie inside any obstacle.
2.Edge-obstacle intersection check: Before adding an edge between two nodes, verify that the straight-line path between them does not intersect or pass through any obstacle.
- DO NOT OVER MAP BOUND
- After code generation, you must review the code to ensure it is syntactically correct, logically coherent, and executable within the expected environment.
- At the top of your response, write an description of the algorithm in curly braces {}, followed by a concise explanation of the planning mechanism in angle brackets <>.
- Both the description and the planning mechanism should be placed outside and above the code block.
- Output the code block containing the implementation only.
⚠️ Do not give additional explanations.


In [32]:
filtered_sorted_algorithms[0].keys()

dict_keys(['operator', 'algorithm_description', 'planning_mechanism', 'code', 'objective', 'time_improvement', 'length_improvement', 'other_inf', 'success_rate'])

In [42]:
result = filtered_sorted_algorithms[3]

code_string = result['code']
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+code_string, planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module
    p = planning_module.Planner(max_iter=5000, step_size=5.0)


In [43]:
p = Planner(max_iter=5000, step_size=5)
# p = planning_module.Planner()

NameError: name 'Planner' is not defined

In [44]:
result = p.plan(map=multi_obs_map)

In [45]:
result2 = p.plan(map=maze_map)

In [46]:
result3 = p.plan(map=narrow_map)

In [47]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal,
                      path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [48]:
visualize_map_shapes(maze_map.grid, obs=maze_map.obstacles, start=maze_map.start, goal=maze_map.goal, path=result2.path, nodes=list(map(lambda x: x.position, result2.nodes)), edges=result2.edges)

In [49]:
visualize_map_shapes(narrow_map.grid, obs=narrow_map.obstacles, start=narrow_map.start, goal=narrow_map.goal, path=result3.path, nodes=list(map(lambda x: x.position, result3.nodes)), edges=result3.edges)

In [306]:
bench2 = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    name="MyRRT"
)
_, avg_res = bench2.run(p.plan)

Map 1, Iteration 1: Time taken: 4.5771 seconds
Map 1, Iteration 2: Time taken: 4.7441 seconds
Map 1, Iteration 3: Time taken: 4.6237 seconds
Map 1, Iteration 4: Time taken: 4.5852 seconds
Map 1, Iteration 5: Time taken: 3.8997 seconds
Map 1, Iteration 6: Time taken: 4.4552 seconds
Map 1, Iteration 7: Time taken: 4.7395 seconds
Map 1, Iteration 8: Time taken: 4.6186 seconds
Map 1, Iteration 9: Time taken: 4.5787 seconds
Map 1, Iteration 10: Time taken: 4.4073 seconds
Map 2, Iteration 1: Time taken: 4.9983 seconds
Map 2, Iteration 2: Time taken: 5.8869 seconds
Map 2, Iteration 3: Time taken: 6.7470 seconds
Map 2, Iteration 4: Time taken: 6.2999 seconds
Map 2, Iteration 5: Time taken: 5.4660 seconds
Map 2, Iteration 6: Time taken: 6.6855 seconds
Map 2, Iteration 7: Time taken: 6.0104 seconds
Map 2, Iteration 8: Time taken: 5.8618 seconds
Map 2, Iteration 9: Time taken: 5.9092 seconds
Map 2, Iteration 10: Time taken: 7.0804 seconds
Map 3, Iteration 1: Time taken: 8.1677 seconds
Map 3, Iter

In [ ]:
def my_dummy_planner(map_array, start, goal):
    return {
        "path": [start, goal],
        "visited": [start],
        "nodes": 2
    }

bench = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    algorithm=p.plan,
    name="MyRRT"
)


df = bench.run()
print(df)

In [17]:
import_string ='''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map

'''

In [9]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [35]:
for method in classic_method:
    if method['algorithm'] == 'BI-RRT*':
        print(method['code'])


class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

    def path_from_root(self):
        path = []
        node = self
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 5.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map) -> PlannerResult:
        import math, random
        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        success_state = False
        extracted_path = []
        nodes = []
        edges = []

        

In [27]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

total_df = pd.DataFrame()

for method in classic_method:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


Map 1
Iteration 1: Time taken: 0.0174 seconds, Success: True
Iteration 2: Time taken: 0.0231 seconds, Success: True
Iteration 3: Time taken: 0.0061 seconds, Success: True
Iteration 4: Time taken: 0.0145 seconds, Success: True
Iteration 5: Time taken: 0.0275 seconds, Success: True
Iteration 6: Time taken: 0.0130 seconds, Success: True
Iteration 7: Time taken: 0.0172 seconds, Success: True
Iteration 8: Time taken: 0.0090 seconds, Success: True
Iteration 9: Time taken: 0.0077 seconds, Success: True
Iteration 10: Time taken: 0.0199 seconds, Success: True
Map 2
Iteration 1: Time taken: 0.1064 seconds, Success: True
Iteration 2: Time taken: 0.2204 seconds, Success: True
Iteration 3: Time taken: 0.1653 seconds, Success: True
Iteration 4: Time taken: 0.1410 seconds, Success: True
Iteration 5: Time taken: 0.1169 seconds, Success: True
Iteration 6: Time taken: 0.1292 seconds, Success: True
Iteration 7: Time taken: 0.0907 seconds, Success: True
Iteration 8: Time taken: 0.0857 seconds, Success: Tr

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
0,RRT,0,1.0,0.015556,248.7,192.016007,0.0,-0.000000,-0.000000,0.000000
1,RRT,1,1.0,0.132758,1099.1,320.395966,0.0,-0.000000,-0.000000,0.000000
2,RRT,2,1.0,0.030499,499.6,160.435752,0.0,-0.000000,-0.000000,0.000000
0,RRT*,0,1.0,0.052024,356.4,171.250880,0.0,-234.434453,10.814269,-68.167482
1,RRT*,1,1.0,0.230198,1051.9,232.617256,0.0,-73.396055,27.396946,-16.539427
2,RRT*,2,1.0,0.100624,656.2,120.078702,0.0,-229.923016,25.154648,-63.945975
0,RRT-Connect,0,1.0,0.005558,82.7,188.456949,0.0,64.270442,1.853521,19.651837
1,RRT-Connect,1,1.0,0.013726,231.9,300.184504,0.0,89.661255,6.308276,28.160032
2,RRT-Connect,2,1.0,0.012776,211.3,152.873662,0.0,58.109748,4.713469,18.375618
0,RRT*-Connect,0,1.0,0.011364,134.2,175.092275,0.0,26.947830,8.813709,9.847091


In [50]:
grouped_avg = total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
alg_name,,,,,,,,,
BI-RRT,1.0,0.966667,0.004420,126.300000,209.299766,-3.333333,88.347410,6.729690,26.183494
BI-RRT*,1.0,0.933333,0.010600,142.266667,172.039175,-6.666667,69.018281,21.824934,21.737138
Improved-RRT*-Connect,1.0,1.000000,1.620073,2580.533333,166.196498,0.000000,-4822.634947,25.286808,-1441.733122
Informed-RRT*,1.0,1.000000,4.898680,3958.500000,156.745682,0.000000,-16810.480676,29.642142,-5037.215774
RRT,1.0,1.000000,0.059604,615.800000,224.282575,0.000000,0.000000,0.000000,0.000000
RRT*,1.0,1.000000,0.127615,688.166667,174.648946,0.000000,-179.251175,21.121954,-49.550962
RRT*-Connect,1.0,1.000000,0.020752,237.533333,192.077747,0.000000,40.379963,13.745243,14.863038
RRT-Connect,1.0,1.000000,0.010687,175.300000,213.838372,0.000000,70.680482,4.291755,22.062496


In [24]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

total_df = pd.DataFrame()

for method in filtered_sorted_algorithms:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


KeyError: 'algorithm'

In [21]:
pop_path = "./path_planning/mobj/results/pops/population_generation_9.json"

with open(pop_path, "r") as f:
    data = json.load(f)
    filtered_sorted_algorithms = sorted(
    [alg for alg in data if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
    )

In [23]:
len(filtered_sorted_algorithms)

19

In [ ]:
filtered_sorted_algorithms = sorted(
    [alg for alg in generated_algorithms if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
)[:10]

In [14]:
filtered_sorted_algorithms = sorted(
    [alg for alg in generated_algorithms if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
)[:10]

In [28]:
# ref_avg 앞에서 선언됨
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

g_total_df = total_df[0:3]

for method in filtered_sorted_algorithms:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['objective']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        try:
            planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        except:
            continue
        res, avg_rest = benchmarker.run(planner.plan)
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        g_total_df = pd.concat([g_total_df, res_df], axis=0)

g_total_df


Map 1
Iteration 1: Time taken: 0.0134 seconds, Success: True
Iteration 2: Time taken: 0.0045 seconds, Success: True
Iteration 3: Time taken: 0.0027 seconds, Success: True
Iteration 4: Time taken: 0.0034 seconds, Success: True
Iteration 5: Time taken: 0.0037 seconds, Success: True
Iteration 6: Time taken: 0.0061 seconds, Success: True
Iteration 7: Time taken: 0.0023 seconds, Success: True
Iteration 8: Time taken: 0.0072 seconds, Success: True
Iteration 9: Time taken: 0.0026 seconds, Success: True
Iteration 10: Time taken: 0.0033 seconds, Success: True
Map 2
Iteration 1: Time taken: 0.0242 seconds, Success: True
Iteration 2: Time taken: 0.0110 seconds, Success: True
Iteration 3: Time taken: 0.0071 seconds, Success: True
Iteration 4: Time taken: 0.0175 seconds, Success: True
Iteration 5: Time taken: 0.0153 seconds, Success: True
Iteration 6: Time taken: 0.0094 seconds, Success: True
Iteration 7: Time taken: 0.0084 seconds, Success: True
Iteration 8: Time taken: 0.0063 seconds, Success: Tr

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
0,RRT,0,1.0,0.015556,248.7,192.016007,0.0,-0.000000,-0.000000,0.000000
1,RRT,1,1.0,0.132758,1099.1,320.395966,0.0,-0.000000,-0.000000,0.000000
2,RRT,2,1.0,0.030499,499.6,160.435752,0.0,-0.000000,-0.000000,0.000000
0,-15.03871,0,1.0,0.004910,77.8,168.619292,0.0,68.438227,12.184773,22.968423
1,-15.03871,1,1.0,0.012151,231.5,292.001134,0.0,90.847167,8.862418,29.026634
2,-15.03871,2,1.0,0.009253,168.0,151.509198,0.0,69.661420,5.563943,22.011215
0,-14.13591,0,1.0,0.005332,79.9,167.508441,0.0,65.722339,12.763293,22.269360
1,-14.13591,1,1.0,0.020020,291.0,296.168309,0.0,84.920012,7.561786,26.988361
2,-14.13591,2,1.0,0.013638,192.6,134.621981,0.0,55.283976,16.089787,19.803150
0,-11.35589,0,1.0,0.005826,71.8,165.061760,0.0,62.545731,14.037500,21.571219


In [31]:
whole_df = pd.concat([total_df, g_total_df], axis=0)

In [ ]:
grouped_avg = g_total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
alg_name,,,,,,,,,
-15.03871,1.0,1.000000,0.008771,159.100000,204.043208,0.000000,76.315605,8.870378,24.668757
-14.13591,1.0,1.000000,0.012997,187.833333,199.432910,0.000000,68.642109,12.138289,23.020290
-11.35589,1.0,1.000000,0.018073,168.566667,200.654919,0.000000,50.243678,10.256080,17.124320
-10.57241,1.0,1.000000,0.010331,152.300000,175.668521,0.000000,73.023453,21.235157,26.154067
-9.55661,1.0,1.000000,0.009778,169.000000,199.482939,0.000000,74.605508,11.184416,24.618536
-9.23875,1.0,1.000000,0.012808,196.033333,209.707866,0.000000,68.177375,6.278594,21.708931
-8.93552,1.0,1.000000,0.017180,216.833333,199.990028,0.000000,57.120232,10.872224,19.310514
-8.86231,1.0,0.966667,0.013297,124.266667,209.325132,-3.333333,69.509462,7.045306,20.595233
-7.55322,1.0,1.000000,0.017622,154.033333,180.891690,0.000000,51.343455,18.307114,19.064459


In [29]:
# alg_name 기준으로 groupby하여 평균값 계산
grouped_avg = whole_df.groupby('alg_name').mean()
print(pd.DataFrame(grouped_avg))

NameError: name 'whole_df' is not defined

In [34]:
col_map = {
    # "map_id": "Map ID",
    "success_rate": "성공률 (Success Rate)",
    "time_avg": "평균 시간 (Time Avg)",
    # "num_nodes_avg": "평균 노드 수 (Num Nodes Avg)",
    "path_length_avg": "평균 경로 길이 (Path Length Avg)",
    "success_improvement": "성공률 개선 (Success Improvement)",
    "time_improvement": "시간 개선 (%)",
    "length_improvement": "경로 개선 (%)",
    "objective_score": "종합 점수 (Objective Score)"
}

df_pretty = grouped_avg.rename(columns=col_map)
df_pretty = df_pretty.round(3)
df_pretty = df_pretty.sort_values("종합 점수 (Objective Score)", ascending=False)
display(df_pretty)

,map_id,성공률 (Success Rate),평균 시간 (Time Avg),num_nodes_avg,평균 경로 길이 (Path Length Avg),성공률 개선 (Success Improvement),시간 개선 (%),경로 개선 (%),종합 점수 (Objective Score)
alg_name,,,,,,,,,
BI-RRT,1.0,0.967,0.004,126.300,209.300,-3.333,88.101,6.730,26.110
RRT-Connect,1.0,1.000,0.012,175.300,213.838,0.000,69.541,4.292,21.721
BI-RRT*,1.0,0.933,0.011,142.267,172.039,-6.667,68.539,21.825,21.593
15.44831,1.0,1.000,0.013,147.633,180.308,0.000,58.762,18.807,21.390
RRT*-Connect,1.0,1.000,0.020,237.533,192.078,0.000,40.091,13.745,14.776
RRT,1.0,1.000,0.055,615.800,224.283,0.000,0.000,0.000,0.000
RRT*,1.0,1.000,0.127,688.167,174.649,0.000,-192.159,21.122,-53.423
623.91658,1.0,1.000,0.141,703.067,217.119,0.000,-252.731,3.369,-75.146
2116.54822,1.0,0.900,0.150,645.033,166.335,-10.000,-271.184,24.860,-81.383


In [23]:
print(filtered_sorted_algorithms[0]['algorithm_description'])

print(filtered_sorted_algorithms[0]['planning_mechanism'])

print(filtered_sorted_algorithms[0]['code'])

The algorithm is a simplified and generalized bidirectional RRT* (BI-RRT*) path planner that builds two trees growing from start and goal positions respectively, incrementally searching for an optimal path in a given environment with obstacles. It samples free points within the map bounds, extends trees toward sampled points while rewiring nearby nodes to minimize path cost, and attempts to connect the two trees when close enough, returning an optimized feasible path.
The planner iteratively samples collision-free points within the environment, extends the nearest node in one tree towards this sample with a fixed step size, rewires nearby nodes to improve path cost, then attempts to connect the other tree to the newly added node. Collision checks on nodes and edges ensure path validity. Upon connecting the two trees, the final path is constructed by concatenating paths from both trees and returned along with the explored nodes and edges.
class Node:
    def __init__(self, position, par

### LM TEST

In [ ]:
from eoh.llm.interface_LLM import InterfaceLLM
from eoh.utils.getParas import Paras
from eoh.methods.eoh.eoh_evolution import Evolution
from eoh.problems.optimization.classic_benchmark_path_planning.utils.prompts import GetPrompts
import re
paras = Paras()


In [170]:
ee = Evolution("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True, GetPrompts())
illm = InterfaceLLM("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True)

- check LLM API
remote llm api is used ...
- check LLM API
remote llm api is used ...


In [171]:
result_data[3]

{'algorithm': 'RRT*-Connect',
 'algorithm_description': 'RRT*-Connect is a hybrid sampling-based motion planning algorithm that merges the strengths of RRT* and RRT-Connect. It grows two trees from the start and goal positions, extending them toward randomly sampled points. At each iteration, it attempts to connect the two trees. Additionally, it performs rewiring to improve path quality, as in RRT*, leading to an asymptotically optimal solution.',
 'planning mechanism': '\n1.\tInitialization\n\t•\tInitialize two trees: one rooted at the start and the other at the goal.\n\t•\tInsert the root nodes (start and goal) into their respective trees.\n2.\tSampling\n\t•\tGenerate a random sample from the configuration space.\n\t•\tWith a small probability (goal bias), use the opposite tree’s root as the sample.\n3.\tExtension\n\t•\tFor the active tree, find the nearest node to the sample.\n\t•\tSteer toward the sample up to a fixed step size and create a new node.\n\t•\tIf the new node is colli

In [176]:
pc = ee.get_prompt([result_data[3]], 'e1')

In [8]:
gcode = illm.get_response(pc)

In [9]:
print(gcode)

{This algorithm implements a Bidirectional Informed RRT* planner with heuristic-based informed sampling and path smoothing. It grows two trees from start and goal states, focusing samples in an ellipsoidal informed set to concentrate the search on promising regions, improving efficiency and solution quality. It applies rewiring to optimize paths and uses a post-processing smoothing step for path smoothness and shorter paths.}  
(A random sample is drawn within an ellipsoid defined by the current best path cost to bias sampling, then nodes are extended towards samples from both trees. The two trees attempt to connect after each iteration, enabling quick discovery of a path. Rewiring improves local paths iteratively. When a path is found, path smoothing refines it by shortcutting unnecessary waypoints.)

```python
from typing import List, Tuple, NamedTuple, Optional
from math import dist, sqrt, cos, sin, acos
import random

# --- Node class ---
class Node:
    def __init__(self, position

In [11]:
c, m, a = ee._extract_alg(gcode)

In [16]:
print(a)

class Node:
    def __init__(self, position: Tuple[float, ...], parent: Optional['Node'] = None, cost: float = 0.0):
        self.position = position        # Tuple[float, ...] for 2D or 3D
        self.parent = parent            # Node or None
        self.cost = cost                # Cost from start node
        self.children: List[Node] = []
        self.valid = True               # For collision checking etc.

    def add_child(self, child: 'Node'):
        self.children.append(child)

    def path(self) -> List[Tuple[float, ...]]:
        node, p = self, []
        while node:
            p.append(node.position)
            node = node.parent
        return p[::-1]
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, goal_sample_rate: float = 0.05, rewire_radius: float = 5.0):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        self.rewire_radius = rewire_radius

    def plan(se